In [ ]:
import requests
import pandas as pd

# stat variables
season_id = '2022-23'
per_mode = 'PerGame'
season_post = 'Playoffs'

# stats url with season_id and per_mode concatenated within
player_info_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
player_info_url_2='https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

response = requests.get(url=player_info_url, headers=headers).json()
response_2 = requests.get(url=player_info_url_2, headers=headers).json()

player_info = response['resultSets'][0]['rowSet']
player_info_2 = response_2['resultSets'][0]['rowSet']

columns_list = [
    "PLAYER_ID",
    "PLAYER_NAME",
    "NICKNAME",
    "TEAM_ID",
    "TEAM_ABBREVIATION",
    "AGE",
    "GP",
    "W",
    "L",
    "W_PCT",
    "MIN",
    "FGM",
    "FGA",
    "FG_PCT",
    "FG3M",
    "FG3A",
    "FG3_PCT",
    "FTM",
    "FTA",
    "FT_PCT",
    "OREB",
    "DREB",
    "REB",
    "AST",
    "TOV",
    "STL",
    "BLK",
    "BLKA",
    "PF",
    "PFD",
    "PTS",
    "PLUS_MINUS",
    "NBA_FANTASY_PTS",
    "DD2",
    "TD3",
    "WNBA_FANTASY_PTS",
    "GP_RANK",
    "W_RANK",
    "L_RANK",
    "W_PCT_RANK",
    "MIN_RANK",
    "FGM_RANK",
    "FGA_RANK",
    "FG_PCT_RANK",
    "FG3M_RANK",
    "FG3A_RANK",
    "FG3_PCT_RANK",
    "FTM_RANK",
    "FTA_RANK",
    "FT_PCT_RANK",
    "OREB_RANK",
    "DREB_RANK",
    "REB_RANK",
    "AST_RANK",
    "TOV_RANK",
    "STL_RANK",
    "BLK_RANK",
    "BLKA_RANK",
    "PF_RANK",
    "PFD_RANK",
    "PTS_RANK",
    "PLUS_MINUS_RANK",
    "NBA_FANTASY_PTS_RANK",
    "DD2_RANK",
    "TD3_RANK",
    "WNBA_FANTASY_PTS_RANK",
]

In [57]:

# points comparison
playoffs_df = pd.DataFrame(player_info, columns=columns_list)
regular_df = pd.DataFrame(player_info_2, columns=columns_list)
#nba_df.to_csv('nba_playoff_season_stats_2023.csv', index=False)
# players_df = nba_df[['PLAYER_NAME', 'PTS']]
# display(players_df.head())

#display(regular_df)
#display(playoffs_df)

# simplify DataFrames ofr data that we need for this analysis
playoffs_df = playoffs_df[['PLAYER_NAME', 'PTS']]
regular_df = regular_df[['PLAYER_NAME', 'PTS']]

# get list of player names from reg season and playoffs
playoff_players = playoffs_df['PLAYER_NAME'].tolist()
regular_players = regular_df['PLAYER_NAME'].tolist()

# players that have stats listed in both playoffs and regular season
names = []
for i in range(len(playoff_players)):
    for j in range(len(regular_players)):
        if playoff_players[i] == regular_players[j]:
            names.append(playoff_players[i])

# list to insert player point differentials as a tuples
stats = []

# loop through names of player that are in the playoffs
for i in range(len(names)):
    reg_stat = regular_df[regular_df['PLAYER_NAME']==names[i]]['PTS'].values[0]
    playoff_stat = playoffs_df[playoffs_df['PLAYER_NAME']==names[i]]['PTS'].values[0]
    stat = (names[i],reg_stat, playoff_stat)
    stats.append(stat)


final_df = pd.DataFrame(stats, columns=['Name','Regular Season pts','Playoff pts'])

display(final_df)
print()

,Name,Regular Season pts,Playoff pts
0,Aaron Gordon,16.3,13.0
1,Aaron Holiday,3.9,0.0
2,Al Horford,9.8,6.7
3,Alex Len,1.7,2.7
4,Amir Coffey,3.4,0.0
...,...,...,...
209,Wesley Matthews,3.4,6.0
210,Xavier Tillman,7.0,8.7
211,Yuta Watanabe,5.6,3.0
212,Zeke Nnaji,5.2,1.3
